# COMP9318-Lab3

## Instructions
1. This note book contains instructions for **COMP9318-Lab3**.

* You are required to complete your implementation in a file `submission.py` provided along with this notebook.

* You are not allowed to print out unnecessary stuff. We will not consider any output printed out on the screen. All results should be returned in appropriate data structures via corresponding functions.

* You can submit your implementation for **Lab3** via following link: https://unswkg.net/submit/ .

* For each question, we have provided you with detailed instructions along with question headings. In case of any problem, you can post your query @ Piazza.

* You are allowed to add other functions and/or import modules (you may have to in this lab), but you are not allowed to define global variables. **Only functions are allowed** in `submission.py`. 

* You should not import unnecessary modules/libraries, failing to import such modules at test time will lead to errors.

* We will provide immediate feedback on your submission. You can access your scores using the online submission portal on the same day. 

* For **Final Evaluation** we will be using a different dataset, so your final scores may vary.  

* You are allowed to submit as many times as you want before the deadline, but **ONLY the latest version will be kept and marked**.

* Submission deadline for this assignment is **20:59:59 on 6th April, 2020 (Sydney Time)**. We will **not** accept any late submissions.

# Quesion 1: Hierarchical Clustering

You are required to implement a hierarchical clustering algorithm.

The input data is a set of unit vectors (i.e., their norm is 1).

In [11]:
import numpy as np
data = np.loadtxt('asset/data_2.txt', dtype = float)
print(data)

[[0.4472136  0.89442719]
 [0.94388373 0.11043153]
 [0.95388373 0.11043153]
 [0.9486833  0.31622777]
 [0.70710678 0.70710678]
 [0.24253563 0.9701425 ]
 [0.99388373 0.11043153]
 [0.91388373 0.11043153]
 [0.92388373 0.11043153]
 [0.93388373 0.11043153]
 [0.96388373 0.11043153]
 [0.94388373 0.11043153]
 [0.95388373 0.11043153]
 [0.9486833  0.31622777]
 [0.70710678 0.70710678]
 [0.24253563 0.9701425 ]
 [0.99388373 0.11043153]
 [0.91388373 0.11043153]]


We use inner product to evaluate the similarity between two vectors.

In [12]:
def dot_product(a, b):
    res = 0
    for i in range(len(a)):
        res += a[i] * b[i]
    return res

print(dot_product(data[0], data[1]))


0.5208905983822762


In [13]:
def complete_link(data, index):
    n = data.shape[0]
    one_dim = []
#     print(data)
    for i in range(n):
        for j in range(i + 1, n):
            one_dim.append(data[i][j])
    max_num = max(one_dim)
    for i in range(n):
        for j in range(i + 1, n):
            if data[i][j] == max_num:
                c = [i, j]
                continue
#     print(np.where(data == max_num)[0])
#     c = np.where(data == max_num)[0]
#     print(c)
    clu = [index[c[0]], index[c[1]]]
#     print(clu)
#     print(index)
    index.remove(clu[0])
    index.remove(clu[1])
    index.append(str(clu[0]) + ' ' +str(clu[1]))
    return c, index
def update_martix(data, c):
    n = data.shape[0]
    new_data = []
    for i in range(n):
        if i not in c:
            temp = []
            for j in range(n):
                if j not in c:
                    temp.append(data[i][j])
            temp.append(min([data[i][c[0]], data[i][c[1]]]))
            new_data.append(temp)
    after_c = []
    for i in range(len(new_data)):
        after_c.append(new_data[i][-1])
    after_c.append(float(1))
    new_data.append(after_c)
    new_data = np.array(new_data)
    return new_data
            
    
def hc1(data, k):
    result_dic = {}
    point_num = len(data)
    sim_martix = []
    result = []
    index = [i for i in range(point_num)]
    for i in range(point_num):
        sim = []
        for j in range(point_num):
            sim.append(dot_product(data[i], data[j]))
        sim_martix.append(sim)
    sim_martix = np.array(sim_martix)
#     print(sim_martix)
    if k != 0:
        if k == 1:
            result = [0] * len(index)
            return result
        if 1 < k <= len(index):
            while len(index) != k:
                c, index= complete_link(sim_martix, index)
                sim_martix = update_martix(sim_martix, c)
        count = 0
#         print(index)
        for i in index:
            if isinstance(i, str):
                split_list = i.split(' ')
                for j in split_list:
                    result_dic[int(j)] = count
                count += 1
        for i in index:
            if isinstance(i, int):
                result_dic[i] = count
                count += 1
        for i in sorted(result_dic):
            result.append(result_dic[i])
    return result
# print(hc(data, 3))



In [22]:
def update_table(sim_table, tem_index, x, y):
    tem_index.remove(x)
    tem_index.remove(y)
    sim_table[x][y] = 0
    
    for i in tem_index:
        if i < x:
            sim_table[i][x] = min(sim_table[i][x], sim_table[i][y])
            sim_table[i][y] = 0
        elif x < i < y:
            sim_table[x][i] = min(sim_table[x][i], sim_table[i][y])
            sim_table[i][y] = 0 
        elif i > y:
            sim_table[x][i] = min(sim_table[x][i], sim_table[y][i])
            sim_table[y][i] = 0
    
    tem_index.append(x)
    return sim_table, tem_index

def hc(data, k):# do not change the heading of the function
#     pass # Replace this line with your implementation...

    dim = len(data)
    result = []
    if k == 1:
        result = [ 0 for i in range(dim)]
        return result    
    
    elif k > 1 and k <= dim:
        sim_table = []
        index_list = [ i for i in range(dim)]
        tem_index = list(np.arange(dim))
        
        for i in range(dim):
            sub_table = []
            for j in range(dim):
                sub_table.append(dot_product(data[i], data[j]))
            sim_table.append(sub_table)
        sim_table = np.array(sim_table)   
        sim_table = np.triu(sim_table, k = 1)
        
        for i in range(dim - k):
            position = np.argmax(sim_table)
            x = position // dim
            y = position % dim
#             print(x, y)
            x, y = int(x), int(y)
            if x in index_list:
                index_list.remove(x)
            if y in index_list:
                index_list.remove(y)
            index_list.append([x, y])          
            sim_table, tem_index = update_table(sim_table, tem_index, x, y)
        
        new_index = []        
        for i in range(len(index_list)):
            if isinstance(index_list[i], int):
                index_list[i] = [index_list[i]]
        while len(index_list) != 1:
            judge = index_list.pop()
            for i in index_list:
                if len(set(i + judge)) < len(i) + len(judge):
                    judge = list(set(i + judge))
                    index_list.remove(i)
            new_index.append(judge)
        new_index.append(index_list[0])       
        
        count = 0
        result = [ 0 for i in range(dim)]
        for i in new_index:
            for num in i:
                result[num] = count
            count += 1

    return result
            


You need to implement the hierarchical clustering algorithm with **complete link**.

The two arguments of ```hc()``` are ```data``` and ```k```, where ```k``` is the number of clusters.

The expected output is a list of ```n``` integers, which correspond to the labels of these vectors. The integer (i.e., label) should in $[0,k)$.

In [24]:
import submission as submission

# for k in range(1, 6):
#     print(hc(data, k))
k = 2
print(hc(data, k))

[1, 2, 3, 4, 1, 6, 5, 0, 2, 0, 4, 2, 3, 4, 1, 6, 5, 0]


We only interesting in the clustering results rather than the labels (for example, ```[1, 0, 2, 1, 0]``` and ```[2, 0, 1, 2, 0]``` are considered to be the same). Thus we will use the average similarity between each vector and its cluster center to evaluate your implementation.

Note: you do not need to implement ```compute_error()``` or include it in your submission.

In [ ]:
def compute_error(data, labels, k):
    n, d = data.shape
    centers = []
    for i in range(k):
        centers.append([0 for j in range(d)])
    
    for i in range(n):
        centers[labels[i]] = [x + y for x, y in zip(centers[labels[i]], data[i])]

    error = 0
    for i in range(n):
        error += dot_product(centers[labels[i]], data[i])
    return error 
    
print(compute_error(data, submission.hc(data, k), k))
print(compute_error(data, hc(data, k), k))